In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow import keras

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [5]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

## Exercises 

### Get CIFAR10 dataset

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [7]:
keras.backend.clear_session()

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

In [9]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal", activation="elu"))    

In [10]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [11]:
optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

#### Callbacks

In [12]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('cifar10_model.m5', save_best_only=True)
run_index = 1

Using Tensorboard to compare lr for 10 epochs. I compared 1e-5,5e-5 and 3e-5. 3e-5 and 3e-5 were good. 5e-5 was best. Check logs for more details in the tensorboard.

In [13]:
import os

In [14]:
root_logdir = os.path.join(os.curdir, "dnn_logs")

In [15]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [16]:
run_logdir = get_run_logdir()

In [17]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_valid, y_valid), 
                    callbacks=[early_stopping_cb, model_checkpoint_cb, tensorboard_cb])

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
44992/45000 [============================>.] - ETA: 0s - loss: 4.4372 - accuracy: 0.1514WARNING:tensorflow:From /Users/sidparmar/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: cifar10_model.m5/assets
45000/45000 [==============================] - 14s 317us/sample - loss: 4.4367 - accuracy: 0.1514 - val_loss: 2.1964 - val_accuracy: 0.1978
Epoch 2/100
45000/45000 [==============================] - 12s 266us/sample - loss: 2.0672 - accuracy: 0.2350 - val_loss: 1.9694 - val_accuracy: 0.2792
Epoch 3/100
45000/45000 [==============================] - 12s 264us/sample - loss: 1.9501 - accuracy: 0.2818 - val_lo

45000/45000 [==============================] - 12s 277us/sample - loss: 1.3098 - accuracy: 0.5284 - val_loss: 1.5216 - val_accuracy: 0.4604
Epoch 35/100
45000/45000 [==============================] - 10s 231us/sample - loss: 1.3020 - accuracy: 0.5313 - val_loss: 1.5783 - val_accuracy: 0.4522
Epoch 36/100
45000/45000 [==============================] - 10s 231us/sample - loss: 1.2906 - accuracy: 0.5361 - val_loss: 1.5372 - val_accuracy: 0.4604
Epoch 37/100
45000/45000 [==============================] - 11s 234us/sample - loss: 1.2850 - accuracy: 0.5399 - val_loss: 1.5591 - val_accuracy: 0.4494
Epoch 38/100
45000/45000 [==============================] - 11s 253us/sample - loss: 1.2780 - accuracy: 0.5416 - val_loss: 1.5310 - val_accuracy: 0.4666
Epoch 39/100
45000/45000 [==============================] - 12s 256us/sample - loss: 1.2655 - accuracy: 0.5457 - val_loss: 1.5724 - val_accuracy: 0.4546
Epoch 40/100
45000/45000 [==============================] - 12s 263us/sample - loss: 1.2600 - a

We only got 47.90% val_accuracy after 50 epochs (early stopping).


Now let's try using Batch Normalization

In [18]:
keras.backend.clear_session()

In [19]:
tf.random.set_seed(42)
np.random.seed(42)

In [20]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())

In [21]:
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
    
model.add(keras.layers.Dense(10, activation="softmax"))

In [22]:
optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

Callbacks

In [23]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10_bn_model.h5", save_best_only=True)

run_index = 1 # increment every time you train the model
import time
run_logdir = os.path.join(os.curdir, "cifar10_bn_logs", time.strftime("run_%Y_%m_%d-%H_%M_%S"))

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [24]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_valid, y_valid), 
                    callbacks=callbacks)

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 23s 515us/sample - loss: 1.8323 - accuracy: 0.3463 - val_loss: 1.6298 - val_accuracy: 0.4128
Epoch 2/100
45000/45000 [==============================] - 17s 371us/sample - loss: 1.6608 - accuracy: 0.4088 - val_loss: 1.5896 - val_accuracy: 0.4336
Epoch 3/100
45000/45000 [==============================] - 17s 376us/sample - loss: 1.5975 - accuracy: 0.4310 - val_loss: 1.5215 - val_accuracy: 0.4560
Epoch 4/100
45000/45000 [==============================] - 17s 377us/sample - loss: 1.5411 - accuracy: 0.4543 - val_loss: 1.4518 - val_accuracy: 0.4736
Epoch 5/100
45000/45000 [==============================] - 17s 379us/sample - loss: 1.5006 - accuracy: 0.4691 - val_loss: 1.4334 - val_accuracy: 0.4844
Epoch 6/100
45000/45000 [==============================] - 17s 381us/sample - loss: 1.4608 - accuracy: 0.4811 - val_loss: 1.4392 - val_accuracy: 0.4904
Epoch 7/100
45000/45000 [==============

In [25]:
model = keras.models.load_model("cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

5000/5000 [==============================] - 1s 145us/sample - loss: 1.3195 - accuracy: 0.5414


[1.3195075395584106, 0.5414]

After 49 epochs (early stopping) we have val_accuracy of 54.14%. Much better with BN.

In [28]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, 
                                 kernel_initializer="lecun_normal", 
                                 activation="selu"))

model.add(keras.layers.Dense(10, activation="softmax"))
optimizer = keras.optimizers.Nadam(lr=7e-4)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10_selu_model.h5", save_best_only=True)
import time
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_selu_logs", time.strftime("run_%Y_%m_%d-%H_%M_%S"))

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100, 
         validation_data=(X_valid_scaled, y_valid),
         callbacks=callbacks)

model = keras.models.load_model("cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 12s 278us/sample - loss: 1.9205 - accuracy: 0.3098 - val_loss: 1.8802 - val_accuracy: 0.3310
Epoch 2/100
45000/45000 [==============================] - 11s 237us/sample - loss: 1.7031 - accuracy: 0.3937 - val_loss: 1.7479 - val_accuracy: 0.3730
Epoch 3/100
45000/45000 [==============================] - 11s 238us/sample - loss: 1.6103 - accuracy: 0.4280 - val_loss: 1.6813 - val_accuracy: 0.4164
Epoch 4/100
45000/45000 [==============================] - 11s 238us/sample - loss: 1.5459 - accuracy: 0.4551 - val_loss: 1.5964 - val_accuracy: 0.4374
Epoch 5/100
45000/45000 [==============================] - 11s 238us/sample - loss: 1.4882 - accuracy: 0.4763 - val_loss: 1.5311 - val_accuracy: 0.4632
Epoch 6/100
45000/45000 [==============================] - 10s 232us/sample - loss: 1.4437 - accuracy: 0.4939 - val_loss: 1.5500 - val_accuracy: 0.4566
Epoch 7/100
45000/45000 [==============

[1.4621454309463502, 0.5054]

We get 50.54% accuracy in 33 epochs. Also, notice that it took 19 epochs to reach the best model, which is much faster than bn model, plus each epoch took only 10 seconds, just like the original model. So it's by far the fastest model to train (both in terms of epochs and wall time).

In [30]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
import time
run_logdir = os.path.join(os.curdir, "my_cifar10_alpha_dropout_logs", time.strftime("run_%Y_%m_%d-%H_%M_%S"))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 13s 290us/sample - loss: 1.8904 - accuracy: 0.3267 - val_loss: 1.7513 - val_accuracy: 0.3918
Epoch 2/100
45000/45000 [==============================] - 10s 229us/sample - loss: 1.6593 - accuracy: 0.4117 - val_loss: 1.7569 - val_accuracy: 0.4128
Epoch 3/100
45000/45000 [==============================] - 11s 235us/sample - loss: 1.5673 - accuracy: 0.4478 - val_loss: 1.6128 - val_accuracy: 0.4516
Epoch 4/100
45000/45000 [==============================] - 10s 232us/sample - loss: 1.5023 - accuracy: 0.4714 - val_loss: 1.6298 - val_accuracy: 0.4458
Epoch 5/100
45000/45000 [==============================] - 11s 234us/sample - loss: 1.4484 - accuracy: 0.4913 - val_loss: 1.5310 - val_accuracy: 0.4768
Epoch 6/100
45000/45000 [==============================] - 10s 232us/sample - loss: 1.4027 - accuracy: 0.5105 - val_loss: 1.5432 - val_accuracy: 0.4676
Epoch 7/100
45000/45000 [==============

[1.509048455619812, 0.5032]

With alph dropout, we get 50.32% accuracy. Which is slightly less than what we got without dropout.  